In [ ]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.applications import EfficientNetB0
from transformers import TFBertModel, BertTokenizer
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
import re
from src.utils import download_images

In [ ]:

# Load datasets
train_df = pd.read_csv('/content/train_subset1.csv')
test_df = pd.read_csv('/content/test_subset1.csv')

from src.utils import download_images
import src.constants
import pandas as pd

# Load the train dataset
train_df = pd.read_csv('/content/train_subset1.csv')

# Download images
download_images(train_df['image_link'].tolist(), 'images/')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 5000/5000 [00:36<00:00, 136.82it/s]


In [ ]:

# Image preprocessing function
def load_and_preprocess_image(img_name, img_dir):
    img_path = f"{img_dir}/{img_name}"
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize the image
    return img


In [ ]:

# Load pre-trained EfficientNet model (without top layers)
base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:

# Function to extract numeric value and unit from entity_value
def extract_numeric_and_unit(value):
    match = re.match(r"([\d\.]+)\s*(\w+)", value)
    if match:
        numeric_part = float(match.group(1))
        unit_part = match.group(2).lower()
        return numeric_part, unit_part
    return None, None


In [ ]:

# Preprocess and extract features for train data
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True # Add this line to handle truncated images

train_df['image_name'] = train_df['image_link'].apply(lambda url: url.split('/')[-1])
train_image_features = np.array([base_model.predict(load_and_preprocess_image(img_name, '/content/images'))
                                 for img_name in train_df['image_name']])
train_image_features = train_image_features.reshape(len(train_df), -1)

# Apply numeric value and unit extraction to train dataset
train_df['numeric_value'], train_df['unit'] = zip(*train_df['entity_value'].map(extract_numeric_and_unit))
train_df['combined_text'] = train_df['entity_name'] + ' ' + train_df['group_id'].astype(str) + ' ' + train_df['unit']


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step

In [18]:
train_df.isna().sum()

,0
image_link,0
group_id,0
entity_name,0
entity_value,0
image_name,0
numeric_value,124
unit,124
combined_text,124


In [19]:
train_df.fillna({'numeric_value': 0, 'unit': 'unknown','combined_text': 'unknown'}, inplace=True)


In [20]:

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Function to encode text using BERT
def encode_text(texts):
    encoded_inputs = tokenizer(texts.tolist(), padding=True, truncation=True, return_tensors='tf')
    outputs = bert_model(encoded_inputs).last_hidden_state[:, 0, :]
    return outputs

# Encode the combined text using BERT for train data
train_text_embeddings = encode_text(train_df['combined_text'])
combined_train_features = np.concatenate([train_image_features, train_text_embeddings.numpy()], axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are i

In [21]:
combined_train_features

array([[ 0.0152552 , -0.04762548, -0.04497867, ..., -0.1852577 ,
         0.21438582,  0.45735312],
       [ 0.01555556, -0.04736982, -0.04598404, ..., -0.06136049,
         0.18430763,  0.3103829 ],
       [ 0.01561032, -0.04746563, -0.04681013, ..., -0.24427491,
         0.23785396,  0.39783487],
       ...,
       [ 0.01531395, -0.04758981, -0.04514997, ..., -0.4055161 ,
         0.3314458 ,  0.4283319 ],
       [ 0.01539389, -0.04755599, -0.04525872, ..., -0.4055161 ,
         0.3314458 ,  0.4283319 ],
       [ 0.01577935, -0.04746736, -0.04647067, ..., -0.23586157,
         0.21478213,  0.44669086]], dtype=float32)

In [39]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(combined_train_features, train_df['numeric_value'], test_size=0.2, random_state=42)



In [40]:
y_train = y_train.values.reshape(-1, 1)
y_val = y_val.values.reshape(-1, 1)

In [41]:


# Initialize and train TabNet model
tabnet_model = TabNetRegressor()
tabnet_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_name=['val'],
    eval_metric=['rmse'],
    max_epochs=50,
    patience=5,
    batch_size=32,
    virtual_batch_size=16
)


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.975580553858835e+16| val_rmse: 17583213.50024|  0:00:05s
epoch 1  | loss: 1.9755805504693696e+16| val_rmse: 17583210.12173|  0:00:09s
epoch 2  | loss: 1.9755805500305236e+16| val_rmse: 17583209.62061|  0:00:12s
epoch 3  | loss: 1.9755805500480216e+16| val_rmse: 17583211.19062|  0:00:15s
epoch 4  | loss: 1.975580548367065e+16| val_rmse: 17583212.47309|  0:00:19s
epoch 5  | loss: 1.975580549586293e+16| val_rmse: 17583209.52432|  0:00:23s
epoch 6  | loss: 1.9755805487851104e+16| val_rmse: 17583211.42173|  0:00:26s
epoch 7  | loss: 1.9755805488261796e+16| val_rmse: 17583210.89399|  0:00:29s
epoch 8  | loss: 1.975580548030004e+16| val_rmse: 17583210.65331|  0:00:33s
epoch 9  | loss: 1.9755805480337868e+16| val_rmse: 17583210.70242|  0:00:37s
epoch 10 | loss: 1.9755805493619092e+16| val_rmse: 17583210.30053|  0:00:40s

Early stopping occurred at epoch 10 with best_epoch = 5 and best_val_rmse = 17583209.52432


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [56]:
y_test_pred = tabnet_model.predict(X_val)

In [57]:
from sklearn.metrics import f1_score

# Define a threshold to classify predictions (you can choose a different method)
threshold = train_df['numeric_value'].median()

# Convert actual values to binary classes based on the threshold
y_true = (y_val >= threshold).astype(int)

# Convert predictions to binary classes based on the same threshold
y_pred = (y_test_pred.flatten() >= threshold).astype(int)

# Calculate F1 score
f1 = f1_score(y_true, y_pred)

print("F1 Score:", f1)


F1 Score: 0.6604152712659076


In [44]:
from src.utils import download_images
import src.constants
import pandas as pd

# Load the train dataset


# Download images
download_images(test_df['image_link'].tolist(), 'images2/')

100%|██████████| 500/500 [00:03<00:00, 131.16it/s]


In [50]:





# Preprocess and extract features for test data
test_df['image_name'] = test_df['image_link'].apply(lambda url: url.split('/')[-1])
test_image_features = np.array([base_model.predict(load_and_preprocess_image(img_name, '/content/images2'))
                                for img_name in test_df['image_name']])
test_image_features = test_image_features.reshape(len(test_df), -1)

# Combine text features for the test data
test_df['combined_text'] = test_df['entity_name'] + ' ' + test_df['group_id'].astype(str)
test_text_embeddings = encode_text(test_df['combined_text'])

# Combine image and text features for the test data
combined_test_features = np.concatenate([test_image_features, test_text_embeddings.numpy()], axis=1)
# Predict the numeric values for the test data
y_test_pred = tabnet_model.predict(combined_test_features)

# Ensure y_test_pred is a 1D array
y_test_pred_flat = y_test_pred.flatten()

# Combine predictions with units for the final output
test_df['predicted_entity_value'] = [f"{pred:.2f}" for pred, unit in zip(y_test_pred_flat, test_df['unit'])]


# Prepare the final output in the required format
test_output = test_df[['index', 'predicted_entity_value']]

# Save the output to a CSV file
test_output.to_csv('sample_test_out2.csv', index=False)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━